# Use TensorBoard to Debug and Analyze Training Jobs in Amazon SageMaker with PyTorch lightning binary segmentation

Automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many jobs that test a range of hyperparameters on your dataset. You choose the tunable hyperparameters, a range of values for each, and an objective metric. You choose the objective metric from the metrics that the algorithm computes. Automatic model tuning searches the hyperparameters chosen to find the combination of values that result in the model that optimizes the objective metric.

## Introduction

This notebook demonstrates the use of the [PyTorch Segmentation models with pretrained backbones](https://github.com/qubvel/segmentation_models.pytorch) - Python library with Neural Networks for Image Segmentation based on [PyTorch](https://pytorch.org/).

The main features of this library are:

- High level API (just two lines to create a neural network)
- 9 models architectures for binary and multi class segmentation (including legendary Unet)
- 124 available encoders (and 500+ encoders from timm)
- All encoders have pre-trained weights for faster and better convergence
- Popular metrics and losses for training routines

This notebook shows how to use `segmentation-models-pytorch` for **binary** semantic segmentation. We will use the [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) (this is an adopted example from Albumentations package [docs](https://albumentations.ai/docs/examples/pytorch_semantic_segmentation/), which is strongly recommended to read, especially if you never used this package for augmentations before).

In [5]:
!pip install sagemaker -U -q


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from sagemaker.pytorch import PyTorch
from datetime import datetime
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/sagemaker_pytorch_binary_segmentation_intro"


# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

print(f"Using SageMaker version {sagemaker.__version__}")
print(f"Using boto3 version {boto3.__version__}")

Using SageMaker version 2.159.0
Using boto3 version 1.26.139


In [7]:
training_dataset_s3_path = "s3://aws-ml-blog/artifacts/amazon-sagemaker-binary-segmentation-intro/oxford-pet-dataset"

In [8]:
!aws s3 ls {training_dataset_s3_path}/

                           PRE annotations/
                           PRE images/
2023-03-05 13:24:33   19173078 annotations.tar.gz
2023-03-05 13:25:57  791918971 images.tar.gz


## Construct a SageMaker training launcher with TensorBoard data configuration

In [9]:
tensorboard_logs_path = "s3://{}/{}/tensorboard_logs".format(bucket, prefix)
tensorboard_logs_path

's3://sagemaker-us-east-1-062083580489/sagemaker/sagemaker_pytorch_binary_segmentation_intro/tensorboard_logs'

In [10]:
from sagemaker.debugger import TensorBoardOutputConfig

LOG_DIR="/opt/ml/output/tensorboard"

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=tensorboard_logs_path,
    container_local_output_path=LOG_DIR
)

In [11]:
metric_definitions = [
    {'Name': 'test_dataset_iou', 'Regex': 'test_dataset_iou: ([0-9.]+).*$'},
    {'Name': 'test_per_image_iou', 'Regex': 'test_per_image_iou: ([0-9.]+).*$'},
]

In [12]:
estimator = PyTorch(entry_point='train.py',
                        source_dir='./code',
                        role=role,
                        framework_version='1.10',
                        py_version='py38',
                        instance_count=1,
                        instance_type='ml.g5.2xlarge',
                        # keep_alive_period_in_seconds=3600,
                        metric_definitions=metric_definitions,
                        tensorboard_output_config=tensorboard_output_config,
                        hyperparameters={
                            'epochs': 3,
                            'arch': "DeepLabV3Plus", ## Unet | FPN | DeepLabV3 | DeepLabV3Plus | Unet \ UnetPlusPlus
                            'tensorboard_logs_path': LOG_DIR
                        })
estimator.fit({"training": training_dataset_s3_path}, logs=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-05-24-13-57-02-069


Using provided s3_resource
2023-05-24 13:57:02 Starting - Starting the training job...
2023-05-24 13:57:18 Starting - Preparing the instances for training......
2023-05-24 13:58:17 Downloading - Downloading input data...
2023-05-24 13:59:02 Training - Downloading the training image..................
2023-05-24 14:01:53 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-24 14:02:23,562 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-24 14:02:23,583 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-24 14:02:23,585 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-24 14:02:23,747 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━

## How to access TensorBoard on SageMaker

You can access TensorBoard by two methods: programatically using the sagemaker.interactive_apps.tensorboard module that generates URL, or using the TensorBoard landing page in the SageMaker console. After you open TensorBoard, SageMaker runs the TensorBoard plugin and automatically finds and loads all training job output data in TensorBoard-compatible file format.

In [21]:
from sagemaker.interactive_apps import tensorboard

region = "us-east-1"
app = tensorboard.TensorBoardApp(region)

print("Navigate to the following URL:")
print(app.get_app_url(training_job_name=estimator.latest_training_job.name))

Navigate to the following URL:
https://d-mcy0pljsdnak.studio.us-east-1.sagemaker.aws/tensorboard/default/data/plugin/sagemaker_data_manager/add_folder_or_job?Redirect=True&Name=pytorch-training-2023-05-24-13-57-02-069
